# **Mentoría Ciencia de datos aplicada a la predicción de licencias médicas y comportamiento de los colaboradores**
Trabajaremos con datos obtenidos de una plataforma de salud ocupacional con el **objetivo de predecir la carga de licencias médicas**. Se propone **realizar un análisis temporal a diferentes escalas**, e identificar las variables asociadas que **expliquen el número de licencias y/o diagnósticos involucrados**.
## **Trabajo Práctico 1: Análisis y Visualización de Datos**

---

# Datos



In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno

sns.set_context("talk")

## Lectura del dataset


In [ ]:
url = "https://raw.githubusercontent.com/MentoriaDiploDatos/Presentacion/main/data.csv"
df = pd.read_csv(url, sep=";")

### #1 **¿Cuáles son las columnas relevantes del dataset? **
Selecciono las columnas más relevantes en base al objetivo:


*   observar si existe un sobredimensionamiento en la cantidad de días solicitados en las licencias médicas
*   el análisis debe ser temporal por lo que son importantes las fechas para realizarlo.

Para lo cual, reviso la info general del DS (dataset) lo que me permite determinar el total de filas,
los tipos de datos de cada columna,
las columnas que no contienen dato (nAn)




In [ ]:
# Reviso la información gral. del dataset
df.info()

In [ ]:
df.columns

De la info recogida en el item 1 se observa que las fechas tienen tipo de dato "object" y lo modificaré para posteriormente realizar análisis de series de tiempo

In [ ]:
df["fecha_creacion"] = pd.to_datetime(df["fecha_creacion"])
df["fecha_inicio"] = pd.to_datetime(df["fecha_inicio"])
df["fecha_fin"] = pd.to_datetime(df["fecha_fin"])
df["fecha_nacimiento"] = pd.to_datetime(df["fecha_nacimiento"])
df.dtypes
df.info()

In [ ]:
# Construyo un dataset con las columnas relevantes
relevant_columns = [
    "fecha_creacion",
    "dias_solicitados",
    "fecha_inicio",
    "fecha_fin",
    "diagnostico",
    "id_diagnostico",
    "fecha_nacimiento",
    "genero",
    "tipo_licencia",
    "estado_civil",
    "dias_aprobados",
    "decision_medica",
    "id_auditor",
    "motivo_rechazo",
    "categoria_diagnostico",
]
dfrelev = df[relevant_columns]
dfrelev[:3]

### 2 *¿Qué tipo de variable es cada una?
Asegurar que tengan el tipo de datos adecuado. Por ejemplo, para el caso de las fechas usar librería datetime para su conversión. En el paso anterior se realizó la transformación del tipo de dato de las fechas


In [ ]:
dfrelev.dtypes
dfrelev.info()

In [ ]:
# ordeno el set de datos por fecha de creación
dfrelev.sort_values(by="fecha_creacion", ascending=True).head(7)

## **3 Exploración del DataSet**
Se revisa el DataSet para revisar si se detectan:
**Valores faltantes**: aquellos datos que no se encuentran presentes en una variable o columna determinada, ya sea porque no se registraron o porque se perdieron.




In [ ]:
# Se revisan los valores nulos (None o NaN)
null_values_count = dfrelev.isnull().sum()
null_values_count

In [ ]:
# len(dfrelev)
# print(len(dfrelev.dias_solicitados))
# print(len(dfrelev.tipo_licencia))

msno.matrix(dfrelev, figsize=(12, 6), fontsize=12, color=[0, 0, 0.2])

# print(dfrelev['dias_solicitados'].isnull().sum())
# print(dfrelev['tipo_licencia'].isnull().sum())
# print(dfrelev['dias_solicitados'].describe() )
# print(dfrelev['tipo_licencia'].describe() )

Para encontrar valores **INCOMPLETOS o CORRUPTOS** se estudia y determinan rangos esperados o ciertos criteros que deben cumplir los valores de cada columna para imputar los correspondientes valores correctos con el método más adecuado o, en caso de ser posible, eliminar las filas con valores faltantes, incompletos o corruptos.

In [ ]:
dfrelev.sort_values(by="fecha_creacion", ascending=True).head(7)

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(
    data=dfrelev[dfrelev.dias_aprobados < 365],
    x="dias_aprobados",
)
plt.show()

### Calculo la cantidad de días de licencia que fueron  efectivamente autorizados

In [ ]:
# 4 CALCULO LA CANTIDAD DE DÍAS EFECTIVAMENTE AUTORIZADOS DE LICENCIA
def days_between(row):
    d1 = pd.to_datetime(row["fecha_inicio"], format="%Y-%m-%d")
    d2 = pd.to_datetime(row["fecha_fin"], format="%Y-%m-%d")
    return abs((d2 - d1).days)


dfrelev["dias_lic_efectiva"] = dfrelev.apply(days_between, axis=1)

## #4 CALCULO LA EDAD
A partir de la fecha de nacimiento calcular la edad del colaborador que solicita la licencia.

In [ ]:
# 4 CALCULO LA EDAD A partir de la fecha de nacimiento calcular la edad del colaborador que solicita la licencia.
def calculate_age(row):
    fecha_nacimiento = pd.to_datetime(row["fecha_nacimiento"])
    fecha_inicio = pd.to_datetime(row["fecha_inicio"])
    return (
        fecha_inicio.year
        - fecha_nacimiento.year
        - (
            (fecha_inicio.month, fecha_inicio.day)
            < (fecha_nacimiento.month, fecha_nacimiento.day)
        )
    )


dfrelev["edad"] = dfrelev.apply(calculate_age, axis=1)
dfrelev.dtypes

In [ ]:
dfrelev[:3]

#6 Realicen un análisis estadístico de cada una de las variables numéricas:
Cantidad de datos, mínimo, máximo, media, mediana, varianza, desviación estándar, cuartil 1, cuartil 3, rango intercuartílico.
    6.A) Elijan una o dos variables categóricas, repetir este análisis y sacar conclusiones.
    6.B) ¿El tiempo es una variable influyente en las distribuciones de estas variables? Ayudarse de gráficos para contestar esta pregunta.

In [ ]:
dfrelev.shape

In [ ]:
dfrelev.describe()

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS SOLICITADOS**
import numpy as np

print(dfrelev["dias_solicitados"].describe())
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS APROBADOS**
print(dfrelev["dias_aprobados"].describe())
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **DIAS LICENCIA EFECTIVA**
print(dfrelev["dias_lic_efectiva"].describe())
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

In [ ]:
# ESTADÍSTICAS DE DATOS NUMÉRICOS **EDAD**
print(dfrelev["edad"].describe())
# dfrelev.apply(np.nanvar, axis=0) FALTA CALCULAR LA VARIANZA..

5) ¿Existen outliers en las variables seleccionadas?

In [ ]:
def filter_extreme_rows(df, column_list):
    # Iterate over the columns and filter out the rows with extreme values
    for column in column_list:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 3 * iqr
        upper_bound = q3 + 3 * iqr
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return df


# filtramos los valores extremos de las columnas numéricas
print(f"""shape antes de filtrar: {dfrelev.shape}""")
dfrelev = filter_extreme_rows(
    dfrelev, ["dias_solicitados", "dias_aprobados", "dias_lic_efectiva", "edad"]
)
print(f"""shape después de filtrar: {dfrelev.shape}""")

### ** 6.A) Elijan una o dos variables categóricas, repetir este análisis y sacar conclusiones.**

In [ ]:
# Se seleccionan las variables categóricas decision_medica y motivo_de_rechazo
dfrelev.decision_medica.unique()

In [ ]:
fig = plt.figure(figsize=(8, 4))  # se genera la figura y se define su tamaño
sns.countplot(
    x=dfrelev["decision_medica"]
)  # gráfico de conteos de la primer columna (decision_medica)
plt.tick_params(axis="x", labelrotation=75)
plt.show()

Las licencias rechazadas son mínimas en comparación con las aprobadas y las que figuran EDITAR_APROBAR.

In [ ]:
dfrelev.motivo_rechazo.unique()

dfrelev.loc[:, "motivo_rechazo"] = dfrelev.motivo_rechazo.replace(
    {
        "Se aprueba la licencia": "lic.aprobada",
        "La documentacion adjuntada no cumple requisitos": "Docum.no cumple req.",
        "Se aprueba por menos dias": "Aprueba menos dias",
        "Diagnostico no valido": "Diagnostico no valido",
        "Motivos formales": "Motivos formales",
        "Ausente a turno": "Ausente a turno",
        "Otros motivos": "Otros motivos",
    }
).fillna(False)

In [ ]:
fig = plt.figure(figsize=(8, 4))  # se genera la figura y se define su tamaño
# sns.countplot(x=dfrelev.motivo_rechazo,orient="v", color='steelblue') # gráfico de conteos de la segunda columna (decision_medica)
sns.set_theme(style="whitegrid")
sns.countplot(x=dfrelev["motivo_rechazo"])
plt.tick_params(axis="x", labelrotation=75)
plt.show()

Se observa que en la columna motivo_rechazo se incluyen valores que no corresponden a un rechazo Ej. licencia aprobada, o el caso de los false que corresponden a datos None o NaN

7) Representen gráficamente cada variable numérica eligiendo el gráfico que considere apropiado. Consideren la posibilidad de generar rangos de datos para su análisis y representación gráfica de las variables. 

    7.A) Repitan los gráficos agrupando por una o dos variables categóricas y sacar conclusiones.

In [ ]:
# Se crea una figura que contenga una cuadricula de 2 filas y 2 columnas para 4 gráficos
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# boxplot de dias_solicitados
sns.boxplot(ax=axes[0, 0], data=dfrelev, x="dias_solicitados", color="steelblue")

# boxplot de dias_aprobados
sns.boxplot(ax=axes[0, 1], data=dfrelev, x="dias_aprobados", color="steelblue")

# boxplot de dias_lic_efectiva
sns.boxplot(ax=axes[1, 0], data=dfrelev, x="dias_lic_efectiva", color="steelblue")

# histograma de edad
sns.histplot(ax=axes[1, 1], data=dfrelev, x="edad", color="steelblue")

In [ ]:
# Ahora repetimos los graficos pero agrupando por género
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# boxplot de dias_solicitados
sns.boxplot(
    ax=axes[0, 0], data=dfrelev, x="dias_solicitados", y="genero", color="steelblue"
)

# boxplot de dias_aprobados
sns.boxplot(
    ax=axes[0, 1], data=dfrelev, x="dias_aprobados", y="genero", color="steelblue"
)

# boxplot de dias_lic_efectiva
sns.boxplot(
    ax=axes[1, 0], data=dfrelev, x="dias_lic_efectiva", y="genero", color="steelblue"
)

# histograma de edad
sns.histplot(ax=axes[1, 1], data=dfrelev, x="edad", hue="genero", color="steelblue")

8) Presenten una tabla de frecuencias y porcentajes para las variables categóricas estado civil y edad.

In [ ]:
# Frecuencias estado_civil
freq_abs_estado_civil = dfrelev.estado_civil.value_counts()
freq_rel_estado_civil = dfrelev.estado_civil.value_counts(normalize=True) * 100

# Se crea un dataframe con las frecuencias absolutas y relativas
df_freq_estado_civil = pd.DataFrame(
    {"freq_abs": freq_abs_estado_civil, "freq_rel": freq_rel_estado_civil}
)
df_freq_estado_civil

# Frecuencias genero
freq_abs_genero = dfrelev.genero.value_counts()
freq_rel_genero = dfrelev.genero.value_counts(normalize=True) * 100

# Se crea un dataframe con las frecuencias absolutas y relativas
df_freq_genero = pd.DataFrame(
    {"freq_abs": freq_abs_genero, "freq_rel": freq_rel_genero}
)
df_freq_genero

tabla_categoricas = (
    pd.crosstab(
        index=dfrelev.estado_civil, columns=dfrelev.genero, normalize=True, margins=True
    ).round(2)
    * 100
)
tabla_categoricas

9) Realicen un gráfico para representar la tabla construida en el punto 8.

In [ ]:
# Dropeamos registros donde All sea menor a 1 porciento
tabla_categoricas = tabla_categoricas[tabla_categoricas["All"] > 1]
# Dropeamos columna All y graficamos
tabla_categoricas.drop("All", axis=1, inplace=False).plot(
    kind="bar",
    figsize=(8, 6),
    rot=0,
    color=["steelblue", "orange", "green"],
    stacked=True,
)

# adjust plot size
plt.rcParams["figure.figsize"] = 14, 6
plt.show()